In [1]:
cd ~/GitHub/Clustering

C:\Users\kalibrr\GitHub\Clustering


In [2]:
%%time
# Test import

import main, cfilter, suffix, scluster, edistance
reload(main)
reload(cfilter)
reload(suffix)
reload(scluster)
reload(edistance)

'import success'

Wall time: 2.55 s


In [3]:
%%time
# Read raw data
from main import read

raw_candidate_industries = read('data/candidate_industries_04_10_2015.txt')
raw_job_industries       = read('data/job_industries_04_10_2015.txt')
raw_relevant_industries  = read('data/industry_of_relevant_users.txt')
raw_educations           = read('data/educ_fields_of_relevant_users.txt')

Wall time: 12 ms


In [4]:
%%time
# Partner with clean data
# Tuple structure: (raw, clean)

from main import cleaners

def pair_with_clean(data, style='industry'):
    return zip(data, map(cleaners[style], data))

candidate_industries     = pair_with_clean(raw_candidate_industries)
job_industries           = pair_with_clean(raw_job_industries)
relevant_industries      = pair_with_clean(raw_relevant_industries)
educations               = pair_with_clean(raw_educations, style='course')

Wall time: 4.36 s


In [5]:
%%time
# Group unique entries for each clean data

from main import smash

def groupunique(datapair):
    
    # get clean part first
    cleanpart = zip(*datapair)[1]
    
    # smash into a unique set
    return smash(cleanpart)

candidates        = groupunique(candidate_industries)
jobs              = groupunique(job_industries)
relevant          = groupunique(relevant_industries)
courses           = groupunique(educations)

Wall time: 212 ms


In [6]:
%%time

import suffix
reload(suffix)

trie = suffix.Trie(courses)
results = []
for item in courses:
    results.append(trie.search(item, 1)[1:])
results


Wall time: 1min 6s


In [7]:
# Test: suffix trie
# with search function using edit distance
# O(knlogn) dynamic complexity

import suffix
reload(suffix)

%time trie = suffix.Trie(courses)
trie.insert('computer science 2')
trie.search('computer science 2', k=4)

Wall time: 344 ms


[('computer science 2', 0),
 ('COMPUTER SCIENCE', 1),
 ('COMPUTER SCIENCES', 1),
 ('COMPUTER SCIENCE W', 1),
 ('COMPUTER SCIEMCE', 2),
 ('COMPUTER SIENCE', 2),
 ('CUMPUTER SCIENCE', 2),
 ('BCS COMPUTER SCIENCE', 4),
 ('BCS COMPUTER SCIENCES', 4)]

In [8]:
# Test: suffix array

reload(suffix)

data = ['gattaca', 'atta', 'aca', 'tac']
%time sa = suffix.SuffixArray(data, ignore_spaces=False)

print '%s %s %25s\t   %10s [ID]\n' % ('Index', 'LCP', 'Concatenated Suffix', 'Original String')

for index in sa:
    print '[%-2d]' % index,
    print '  %d' % sa.lcp[index],
    print '\t%-25s' % sa.string[index:],

    string_index = sa.G[index]
    string_data = sa.data[string_index]

    print '\t%10s [%d]' % (string_data, string_index)

Wall time: 0 ns
Index LCP       Concatenated Suffix	   Original String [ID]

[20]   0 	$                         	       tac [3]
[16]   0 	$tac$                     	       aca [2]
[12]   0 	$aca$tac$                 	      atta [1]
[7 ]   0 	$atta$aca$tac$            	   gattaca [0]
[15]   0 	a$tac$                    	       aca [2]
[11]   1 	a$aca$tac$                	      atta [1]
[6 ]   1 	a$atta$aca$tac$           	   gattaca [0]
[18]   1 	ac$                       	       tac [3]
[13]   2 	aca$tac$                  	       aca [2]
[4 ]   3 	aca$atta$aca$tac$         	   gattaca [0]
[8 ]   1 	atta$aca$tac$             	      atta [1]
[1 ]   4 	attaca$atta$aca$tac$      	   gattaca [0]
[19]   0 	c$                        	       tac [3]
[14]   1 	ca$tac$                   	       aca [2]
[5 ]   2 	ca$atta$aca$tac$          	   gattaca [0]
[0 ]   0 	gattaca$atta$aca$tac$     	   gattaca [0]
[10]   0 	ta$aca$tac$               	      atta [1]
[17]   2 	tac$                      	  

In [20]:
# Test: suffix array
# Concatenates all data into one big string
# O(nlogn) Complexity

reload(suffix)
%time sa = suffix.SuffixArray(jobs, ignore_spaces = True)


for i in sa[len(jobs) + 20 : len(jobs) + 30]:
    
    ID = sa.G[i]    
    print 'LCP:', sa.lcp[i], sa.string[i:i+30] + '...',
    print '\t', ID, sa.data[ID]

Wall time: 234 ms
LCP: 6 ATIONS$SUGARPROCESSING$SYSTEMS... 	154 INVESTIGATIONS
LCP: 6 ATIONS$TRADE$DRIVERS$AFFAIRS$G... 	149 RESTAURANT SERVICE OPERATIONS
LCP: 6 ATIONS$CONSULTING$JOBWAYS$VARI... 	75 OPERATIONS
LCP: 5 ATIONTECHNOLOGYCOMPANY$BUSINES... 	130 INFORMATION TECHNOLOGY COMPANY
LCP: 17 ATIONTECHNOLOGYCONSULTINGFIRM$... 	201 INFORMATION TECHNOLOGY CONSULTING FIRM
LCP: 15 ATIONTECHNOLOGYORCOMPUTER$PROC... 	39 INFORMATION TECHNOLOGY OR COMPUTER
LCP: 16 ATIONTECHNOLOGYOUTSOURCING$MAN... 	347 INFORMATION TECHNOLOGY OUTSOURCING
LCP: 15 ATIONTECHNOLOGYSERVICES$HOSPIT... 	147 INFORMATION TECHNOLOGY SERVICES
LCP: 16 ATIONTECHNOLOGYSOFTWARE$PRODUC... 	17 INFORMATION TECHNOLOGY SOFTWARE
LCP: 17 ATIONTECHNOLOGYSOTFWARE$COMPUT... 	176 INFORMATION TECHNOLOGY SOTFWARE


In [21]:
# Test: similarity matrix
# returns

reload(suffix)
%time sm = sa.similarity_graph()

sm[:10]

Wall time: 12 ms


[((34, 294), 12),
 ((112, 297), 4),
 ((120, 217), 32),
 ((321, 357), 2),
 ((30, 287), 4),
 ((34, 153), 8),
 ((41, 57), 4),
 ((336, 337), 30),
 ((191, 232), 508),
 ((157, 349), 8)]

In [31]:
# Test: Lattice Spanning
# Spans based on strict substring similarity count
# Main purpose is to reduce noise

import scluster
reload(scluster)

print  'Clustering %d items' % len(courses)
%time cluster = scluster.lattice_spanning(courses, min_common=10)

cluster

Clustering 2186 items
Wall time: 4.33 s


[[''],
 ['MATHEMTICS'],
 ['PAMANTASAN LUNGSOD NG MAKATI'],
 ['I COMMERCE'],
 ['POWERPOINT'],
 ['CHINA BEIJING'],
 ['CUM LAUDE'],
 ['GEOLOGY'],
 ['SCRIPT IMG SRC SEARCH'],
 ['BANLIC CABUYAO LAGUNA'],
 ['TOURISM PLANNING'],
 ['DANCE'],
 ['DIGITAL FILMMAKING', 'FILMMAKING'],
 ['ST FRANCIS DIVINE'],
 ['PHYSICOLOGY'],
 ['THEOLOGY PROPER'],
 ['ECONOMICS'],
 ['BEED SPED'],
 ['AREA OF'],
 ['CULINARYS ARTS'],
 ['BACHEL'],
 ['FRENCH STUDIES'],
 ['A'],
 ['MIDWIFERY'],
 ['MUSIC'],
 ['CFA SCULPTURE'],
 ['STANDARD ENGLISH', 'STANDARD ENGLIS THEOLOGY'],
 ['ALGORITHMS'],
 ['COMMERCE'],
 ['2ND SEM'],
 ['INSURANCE UNDERWRITER'],
 ['MACHINING NCLL'],
 ['198 1984'],
 ['CERTIFIED MEDICATION AIDE'],
 ['PROGRESS'],
 ['HRIM'],
 ['MEDIA'],
 ['CLASSICAL MUSIC', 'CLASSICAL LANGUAGE'],
 ['CBAC'],
 ['NURSING REGISTERED NURSE'],
 ['AMMING'],
 ['MIDDLE EAST STUDIES'],
 ['B'],
 ['BACHELORB'],
 ['MARIA CLARA HIGH SCHOOL'],
 ['SEAMAN'],
 ['ENGLISH TEACHING', 'HISTORY SPANISH TEACHING'],
 ['CELL BIOLOGY'],
 ['INTERNET W

In [32]:
# Test: Dense Spanning
# Spans based on an expected substring similarity count
# Main purpose is to reduce noise

import scluster
reload(scluster)

print 'Clustering %d items' % len(courses)
%time cluster = scluster.dense_spanning(courses, min_common=10)

cluster

Clustering 2186 items
Wall time: 5.12 s


[['LINGUISTICS',
  'HOSPITALITY MANANGEMENT',
  'FOODSERVICE MANAGEMENT',
  'ECONOMICS HONORS',
  'COMPUTER SCIENCE INFORMATION TECHNOLOGY',
  'COMPUTER ELECTRONIC TECHNICIAN',
  'MULTIMEDIA ARTS SCIENCES',
  'SOCIAL SCIENCE HISTORY',
  'ENGINEERING',
  'BROADCASTING',
  'ELECTRONIC COMMUNICATION ENGINEER',
  'TRANSLATION',
  'CHILDHOOD STUDIES',
  'BUSINESS INFORMATION SYSTEM',
  'DENTAL MEDICINE',
  'COMPUER SECRETATIAL OFFICE MANAGEMENT',
  'CULINARY FOOD BEVERAGES',
  'HUMANISTIC SOCIOLOGY',
  'EDUCATION SPORTS RECREATION MANAGEMENT',
  'COMPUTER HARDWARE SERVICING NC 11',
  'MEDICAL SUPPORT ASSISTANT',
  'ELECTRICAL COMPUTER ENGINEERING SIX YEAR CURRICULUM',
  'NURSING ASSISTANT',
  'SOCIOLOGY ECONOMICS',
  'HEALTHCARE',
  'CUSTOMS ADMINISTRAION',
  'AGRICULTURE',
  'HOTEL RETAURANT MANAGEMENT',
  'MEDICAL TRANSCRIPTION SERVICES',
  'INFORMATION TECNOLOGY',
  'FOOD SERVICE',
  'MARKETING ADVERTISING',
  'INTER DISCIPLINARY STUDIES',
  'N SOCIOLOGY',
  'LEGAL MANAGEMENT',
  'EXECUT

In [36]:
%%time

# Test: Lattice clustering based on variance
# Results: not adequate

import scluster
reload(scluster)

print 'Clustering %d items' % len(courses)

cl = scluster.lattice_spanning(courses, min_common = 3)
noise = []

def variance(data):
    if len(data) == 1: return 0
    mean = float(sum(data)) / len(data)
    var = sum(map(lambda x: (x - mean) ** 2, data)) / (len(data) - 1)
    print 'variance:', var, data[:10]
    return var

while variance(map(len,cl) + map(len,noise)) > 20:
    cl.sort(key=len, reverse=True)
    noise.extend(cl[1:])
    cl = scluster.spanning_forest(cl[0], n_clusters=len(cl[0]) / 8)


Clustering 2186 items
variance: 132194.314286 [1, 1, 1, 1, 2152, 1, 1, 1, 1, 1]
variance: 11490.990252 [1867, 3, 3, 3, 3, 2, 2, 2, 2, 2]
variance: 4826.64053345 [1608, 4, 4, 3, 3, 3, 3, 2, 2, 2]
variance: 2508.80995755 [1359, 4, 4, 4, 4, 3, 3, 3, 3, 3]
variance: 1386.62315809 [1120, 9, 6, 6, 5, 5, 4, 4, 4, 4]
variance: 769.091273156 [896, 17, 9, 5, 5, 4, 4, 4, 4, 4]
variance: 419.529089272 [696, 10, 8, 7, 5, 5, 5, 5, 5, 5]
variance: 242.514687571 [548, 20, 9, 9, 5, 4, 4, 4, 3, 3]
variance: 131.591604912 [413, 24, 15, 8, 6, 3, 3, 3, 3, 3]
variance: 62.9844735438 [286, 44, 12, 7, 6, 3, 3, 3, 3, 2]
variance: 22.0879489929 [131, 98, 14, 6, 2, 2, 2, 2, 2, 2]
variance: 18.6131510377 [112, 4, 2, 1, 1, 1, 1, 1, 1, 1]
Wall time: 27.1 s


In [40]:
# Test: Recursive lattice clustering
# Span a subclsuter until pairwise
# edit radius becomes ideal.
# Uses lattice spanning (strict substring length) technique

import scluster
reload(scluster)

print 'Clustering %d items' % len(courses)
%time double_lattice = scluster.double_cluster(courses, min_common=5, step=2, eps=0.4, algorithm='lattice')

double_lattice

Clustering 2186 items
Wall time: 1min 27s


[[[[[[[[[[[[['ELECTRONIC COMMUNICATION ENGINEER',
             'ECT ELECTRONIC COMMUNICATION TECHNICIAN',
             'ELECTRONIC COMMUNICATION ENGINEERING TECHNOLOGY',
             'ELECTRONIC COMMUNICATION ENGINEERING',
             'ELECTRONIC COMMUNICATION ENGINEERINH',
             'ELECTRONIC COMMUNICATION ENGINEER CABLE TELEVISION SATELLITE BROADCAST',
             'MECHANICAL PRODUCTION ENGINEERING TECHNOLOGY',
             'ELECTRONIC COMMUNICATION TECHNICIAN'],
            ['ELECTRONICS COMMUNICATIONS ENGINEERING',
             'ELECTRINOCS COMMUNICATIONS ENGINEERING',
             'ELECTRONICS COMMUNICATIONS ENGINEERING MANAGEMENT TECHNOLOGY',
             'ELECTRONIC COMMUNICATIONS ENGINEERING',
             'ELECTRONICS COMMUNICATIONS TECHNOLOGY',
             'ELECTRONICS COMMUNICATIONS ENGINEERING TECHNOLOGY',
             'COMPUTER ENGINEERING MANAGEMENT TECHNOLOGY'],
            [['NETWORKING TELECOMMUNICATIONS TECHNOLOGY',
              'ELECTRONICS TELECOMMUNICATION

In [54]:
# Test: Recursive dense spanning
# Span a subcluster until pairwise
# edit radius becomes ideal.
# Uses dense spanning (expected substring length) technique

import scluster
reload(scluster)

print 'Clustering %d items' % len(courses)

%time double_dense = scluster.double_cluster(courses, min_common=5, step=0.8, eps=0.4, algorithm='dense')

double_dense

Clustering 2186 items
Wall time: 2min 2s


[[[[[[[[[[[[[[[[['INTERNATIONAL COMMUNICATIONS',
                 'EDUCATIONAL COMMUNICATION',
                 'VISUAL COMMUNICATIONS',
                 'ORGANIZATIONAL COMMUNICATION',
                 'VISUAL COMMUNICATION',
                 'ORGANIZATIONAL COMMUNICATIONS'],
                [[[[['CALLCENTER SERVICES LADDERIZED TO MASS COMMUNICATIONS UNDERGRAD'],
                    ['BROADCAST JOURNALISM MASS COMMUNICATIONS']],
                   ['BROADCAST JOURNALISM']],
                  ['MASS COMMUNICATIONS']],
                 ['MASS COMMUNICATION CISM', 'MASS COMMUNICATION BROADCAST'],
                 ['GENERAL MASS COMMUNICATION'],
                 ['MASS COMMUNICATION'],
                 ['ENGLISH MASS COMMUNICATION']],
                [['WEB APPLICATION DEVELOPMENT', 'WEB APPLICATION DEVELOMENT'],
                 ['INTERNATIONAL COOKERY BARISTA'],
                 ['INTERNATIONAL COOPERATION DEVELOPMENT 2012 2014'],
                 ['INTARNATIONAL COOKERY']],
           

In [60]:
# Test: Labelling recursive heirarchy strategy
# Label heirarchy based on a cluster representative
# Calculated via minimizing edit distance to candidate representative

import scluster
reload(scluster)
reload(edistance)

%time labels = scluster.label_heirarchy(double_dense)

------------------+ INTERNATIONAL COMMUNICATIONS
------------------+ EDUCATIONAL COMMUNICATION
------------------+ VISUAL COMMUNICATIONS
------------------+ ORGANIZATIONAL COMMUNICATION
------------------+ VISUAL COMMUNICATION
------------------+ ORGANIZATIONAL COMMUNICATIONS
-----------------+ ORGANIZATIONAL COMMUNICATIONS
----------------------+ CALLCENTER SERVICES LADDERIZED TO MASS COMMUNICATIONS UNDERGRAD
---------------------+ CALLCENTER SERVICES LADDERIZED TO MASS COMMUNICATIONS UNDERGRAD
----------------------+ BROADCAST JOURNALISM MASS COMMUNICATIONS
---------------------+ BROADCAST JOURNALISM MASS COMMUNICATIONS
--------------------+ CALLCENTER SERVICES LADDERIZED TO MASS COMMUNICATIONS UNDERGRAD
---------------------+ BROADCAST JOURNALISM
--------------------+ BROADCAST JOURNALISM
-------------------+ CALLCENTER SERVICES LADDERIZED TO MASS COMMUNICATIONS UNDERGRAD
--------------------+ MASS COMMUNICATIONS
-------------------+ MASS COMMUNICATIONS
------------------+ CALLCENTE

ZeroDivisionError: float division by zero

In [58]:
labels

NameError: name 'labels' is not defined

In [92]:
import suffix
reload(suffix)

trie = suffix.Trie(courses)

In [93]:
%%time
neighbors = trie.nearest_neighbor('COMPUTERS')
c = 0
res = []
while True:
    neighbor, dist = neighbors.next()
    if dist > 30: break
    c += 1
    res.append((neighbor, dist))
print len(res)

2027
Wall time: 1.32 s


In [ ]:
%%time

import gcluster
reload(suffix)
reload(gcluster)

gcluster.gravitational_clustering(courses)

Creating mapper...
Creating trie...
Done craeting trie.
Making clusters...